In [1]:
import pandas as pd

In [3]:
df_reviews = pd.read_json("yelp_dataset/yelp_academic_dataset_review.json", lines=True, nrows=100)
df_business = pd.read_json("yelp_dataset/yelp_academic_dataset_business.json", lines=True, nrows=100)

In [2]:
#